<a href="https://colab.research.google.com/github/seorinchoi/homework/blob/main/CNN_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

print(torch.__version__)


2.1.0+cu118


In [ ]:
batch_size= 256
learning_rate = 0.0002
num_epoch = 40

#파라미터 설정

In [ ]:
mnist_train = dset.MNIST(root="../",train=True, transform=transforms.ToTensor(),target_transform=None,download=True)
mnist_test = dset.MNIST(root="../",train=False, transform=transforms.ToTensor(),target_transform=None,download=True)

#root=저장위치
#train=학습여부

In [ ]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

print(len(mnist_train),len(mnist_test))

train_loader=DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=2,drop_last=True)
test_loader=DataLoader(mnist_test,batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000
60000 10000


In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,padding=1),
        #커널사이즈가 3X3이므로 패딩을 추가한다.
        nn.ReLU(),
        nn.Dropout2d(0.8), #0.8.짜리 드롭아웃을 넣는다.
        nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Dropout2d(0.8),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Dropout2d(0.8),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.fc_layer = nn.Sequential(
          nn.Linear(64*7*7, 256),  # 첫 번째 레이어, 256개의 노드
          nn.ReLU(),
          nn.Dropout(0.8),
          nn.Linear(256, 256),  # 두 번째 레이어, 256개의 노드
          nn.ReLU(),
          nn.Dropout(0.8),
          nn.Linear(256, 10)  # 출력 레이어, 10개의 노드
    )

  def forward(self,x):
    out=self.layer(x)
    out = out.view(batch_size,-1)

    out=self.fc_layer(out)
    return out

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model=CNN().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)




cpu


In [ ]:
loss_arr = []
for i in range (num_epoch):
  for j, [image,label] in enumerate(train_loader):
    x=image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output=model.forward(x)
    loss=loss_func(output,y_)
    loss.backward()
    optimizer.step()


    if j%6000 == 0:
      print(loss)
      loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3363, grad_fn=<NllLossBackward0>)


KeyboardInterrupt: ignored

In [ ]:
plt.plot(loss_arr)
plt.show()

In [ ]:
correct = 0
total = 0
#맞은 개수, 전체 개수를 저장할 변수


with torch.no_grad():
  for image, label in test_loader:
    x = image.to(device)
    y_ = label.to(device)

    output = model.forward(x)

    _,output_index = torch.max(output,1)

    total += label.size(0) #배치사이즈나 드롭 라스트의 영향으로 데이터가 잘릴 수도 있음

    correct += (output_index == y_).sum().float()
    #일치하면 코렉트에 더함

print("Accuracy of Test data:{}%".format(100*correct/total))